# Tabulated EOS IllinoisGRMHD: $\tau_{\rm floor}$ estimation

In this notebook we perform an estimate of the value of $\tau_{\rm floor}$, i.e. the minimum value allowed for the conserved energy variable $\tau$ (see definition below) during a tabulated equation of state (EOS) run with `IllinoisGRMHD`.

The definition of the conserved energy variable $\tilde{\tau}$ can be read from equation (7) of the [`IllinoisGRMHD` release paper](https://arxiv.org/pdf/1501.07276.pdf) (see also equation (37) in [Duez *et al.* 2005](https://arxiv.org/pdf/astro-ph/0503420.pdf))

$$
\newcommand{\rhob}{\rho_{\rm b}}
\newcommand{\rhostar}{\rho_{\star}}
\tilde{\tau} = \alpha^{2}\sqrt{\gamma}T^{00} - \rhostar\ ,
$$

where $\alpha$ is the lapse function, $\gamma = \det\left(\gamma_{ij}\right)$, with $\gamma_{ij}$ the physical spatial metric, $\rhostar \equiv \alpha\sqrt{\gamma}\rhob u^{0}$ is the conserved density and $\rhob$ the baryon density, $T^{\mu\nu}$ is the GRMHD energy-momentum tensor,

$$
\newcommand{\rhob}{\rho_{\rm b}}
T^{\mu\nu} = \left(\rhob h + b^{2}\right)u^{\mu}u^{\nu} + \left(P + \frac{b^{2}}{2}\right)g^{\mu\nu} - b^{\mu}b^{\nu}\ ,
$$

where $u^{\mu}$ is the fluid 4-velocity, $h = 1 + \epsilon + P/\rhob$ is the specific enthalpy, $\epsilon$ is the specific internal energy, $g^{\mu\nu}$ is the inverse of the physical spacetime metric $g_{\mu\nu}$, and $b^{\mu}$ the magnetic field measure by an observer comoving with the fluid normalized by $\sqrt{4\pi}$.

Following the standard definition of tilde variables in the literature, which are sometimes refered to as "densitized variables", we have that the "undensitized" conserved energy, $\tau$, and density, $D$, are given by

$$
\begin{align}
D &= \frac{\rhostar}{\sqrt{\gamma}} = \alpha u^{0}\rhob = W\rhob\ ,\\
\tau &= \frac{\tilde{\tau}}{\sqrt{\gamma}} = \alpha^{2}T^{00} - D\ ,
\end{align}
$$

where $W = \alpha u^{0}$ is the Lorentz factor.

To perform an estimation of $\tau_{\rm floor}$, we can choose flat space with zero velocities, i.e. $W=1$, and zero magnetic fields, i.e. $b^{\mu} = 0$. Since in flat space $\alpha=1$, we also know that $u^{0}=1$. This implies that

$$
T^{00}_{\rm flat} = \rhob h + P\ .
$$

Also note that $D = W \rhob = \rhob$ in this case. Then

$$
\begin{align}
\tau &= \alpha^{2}T^{00} - D\\
     &= \rhob h + P - \rhob\\
     &= \rhob\left(1 + \epsilon + \frac{P}{\rhob}\right) + P - \rhob\\
     &= \rhob + \rhob\epsilon + P + P - \rhob\\
     &= 2P + \rhob\epsilon
\end{align}
$$

Now we can simply use the EOS table to find the minimum allowed value of $\tau$.

In [5]:
import numpy as np
import h5py

# Read in rho, P, and eps from the EOS table
with h5py.File("/Users/werneck/eos_tables/SLy4_3335_rho391_temp163_ye66.h5","r") as table:
    # Table keys for rho, P, and eps are "logrho",
    # "logpress", and "logenergy", respectively.
    # These all correspond to log10() of the
    # respective quantity. Also, note that the
    # specific internal energy is given in the
    # table as
    #
    # log10(eps + eps_0),
    #
    # where eps_0 is the entry "energy_shift",
    # which is a scalar.
    rhob = 10**np.array(table["logrho"   ])[:]
    P    = 10**np.array(table["logpress" ])[:]
    eps  = 10**np.array(table["logenergy"])[:] - table["energy_shift"][0]

# At this stage, the quantities we just read are given in cgs
# units. This is not particularly interesting for us because
# we use units in which G = c = Msun = 1 in IllinoisGRMHD.
# So let us first convert to the appropriate units.
import astropy.constants as constants
# Speed of light [L]^{1} [T]^{-1}
c = constants.c.cgs.value
# Gravitational constant [L]^{3} [M]^{-1} [T]^{-2}
G = constants.G.cgs.value
# Mass of the sun [M]^{1}
M = constants.M_sun.cgs.value
# Now compute the unit of length
L_units = G*M/c**2
# Then the unit of time
T_units = L_units/c
# Then the unit of density
D_units = M/L_units**3
# Then the unit of pressure
P_units = M/L_units/T_units**2
# Finally the units of specific internal energy (erg/g)
E_units = c**2

# Now convert to M=c=Msun=1 units
rhob /= D_units
P    /= P_units
eps  /= E_units

Now compute

$$
\tau = 2P + \rhob\epsilon\ ,
$$

and find the minimum value allowed by the table.

In [9]:
tau = 2*P + rhob*eps
print("Minimum allowed tau (flat space, zero velocities, zero magnetic fields): %.15e"%(tau.min()))

Minimum allowed tau (flat space, zero velocities, zero magnetic fields): -3.106926148724047e-10
